In [1]:
import numpy as np
from embiggen.embedders.layers import GraphAttention
from ensmallen_graph.datasets.linqs import Cora
from ensmallen_graph.datasets.linqs.parse_linqs import get_words_data
from plot_keras_history import plot_history
from tqdm.keras import TqdmCallback

In [2]:
cora = Cora()
features = get_words_data(cora)
cora = cora.filter_from_names(node_type_name_to_filter=['Word']).remove_edge_weights().remove_edge_types()
features = features.loc[cora.get_node_names()]
#cora = cora.generate_new_edges_from_node_features(features.values, neighbours_number=3, max_degree=3)
cora

Extracting words features:   0%|          | 0/4141 [00:00<?, ?it/s]

The undirected graph Cora has 2708 nodes and 5278 unweighted edges, of which none are self-loops. The graph is sparse as it has a density of 0.00144 and has 78 connected components, where the component with most nodes has 2485 nodes and the component with the least nodes has 2 nodes. The graph median node degree is 3, the mean node degree is 3.90, and the node degree mode is 2. The top 5 most central nodes are 35 (degree 168), 6213 (degree 78), 1365 (degree 74), 3229 (degree 65) and 910 (degree 44). The hash of the graph is 9982f2a4ae22d17d.

In [3]:
A = cora.add_selfloops().get_dense_binary_adjacency_matrix()
nodes_number = cora.get_nodes_number()

In [4]:
train_graph, validation_graph = cora.node_label_holdout(0.8, use_stratification=True)

In [5]:

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.initializers import Zeros
from tensorflow.keras.layers import Layer, Dropout,Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model


class GATLayer(Layer):

    def __init__(self, att_embedding_size=8, head_num=8, dropout_rate=0.5, l2_reg=0, activation=tf.nn.relu,
                 reduction='concat', use_bias=True, seed=1024, **kwargs):
        if head_num <= 0:
            raise ValueError('head_num must be a int > 0')
        self.att_embedding_size = att_embedding_size
        self.head_num = head_num
        self.dropout_rate = dropout_rate
        self.l2_reg = l2_reg
        self.activation = activation
        self.act = activation
        self.reduction = reduction
        self.use_bias = use_bias
        self.seed = seed
        super(GATLayer, self).__init__(**kwargs)

    def build(self, input_shape):

        X, A = input_shape
        embedding_size = int(X[-1])
        self.weight = self.add_weight(name='weight', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                      dtype=tf.float32,
                                      regularizer=l2(self.l2_reg),
                                      initializer=tf.keras.initializers.glorot_uniform())
        self.att_self_weight = self.add_weight(name='att_self_weight',
                                               shape=[1, self.head_num,
                                                      self.att_embedding_size],
                                               dtype=tf.float32,
                                               regularizer=l2(self.l2_reg),
                                               initializer=tf.keras.initializers.glorot_uniform())
        self.att_neighs_weight = self.add_weight(name='att_neighs_weight',
                                                 shape=[1, self.head_num,
                                                        self.att_embedding_size],
                                                 dtype=tf.float32,
                                                 regularizer=l2(self.l2_reg),
                                                 initializer=tf.keras.initializers.glorot_uniform())

        if self.use_bias:
            self.bias_weight = self.add_weight(name='bias', shape=[1, self.head_num, self.att_embedding_size],
                                               dtype=tf.float32,
                                               initializer=Zeros())
        self.in_dropout = Dropout(self.dropout_rate)
        self.feat_dropout = Dropout(self.dropout_rate, )
        self.att_dropout = Dropout(self.dropout_rate, )
        # Be sure to call this somewhere!
        super(GATLayer, self).build(input_shape)

    def call(self, inputs, **kwargs):

        X, A = inputs
        X = self.in_dropout(X)  # N * D
        if K.ndim(X) != 2:
            raise ValueError(
                "Unexpected inputs dimensions %d, expect to be 2 dimensions" % (K.ndim(X)))

        features = tf.matmul(X, self.weight, )  # None F'*head_num
        features = tf.reshape(
            features, [-1, self.head_num, self.att_embedding_size])  # None head_num F'
        attn_for_self = tf.reduce_sum(
            features * self.att_self_weight, axis=-1, keepdims=True)  # None head_num 1
        attn_for_neighs = tf.reduce_sum(
            features * self.att_neighs_weight, axis=-1, keepdims=True)

        dense = tf.transpose(
            attn_for_self, [1, 0, 2]) + tf.transpose(attn_for_neighs, [1, 2, 0])

        dense = tf.nn.leaky_relu(dense, alpha=0.2)
        mask = -10e9 * (1.0 - A)
        dense += tf.expand_dims(mask, axis=0)  # [?,8,8], [1,?,2708]

        self.normalized_att_scores = tf.nn.softmax(
            dense, axis=-1, )  # head_num None(F) None(F)

        features = self.feat_dropout(features, )
        
        self.normalized_att_scores = self.att_dropout(
            self.normalized_att_scores)

        result = tf.matmul(self.normalized_att_scores,
                           tf.transpose(features, [1, 0, 2]))  # head_num None F D   [8,2708,8] [8,2708,3]
        result = tf.transpose(result, [1, 0, 2])  # None head_num attsize

        if self.use_bias:
            result += self.bias_weight

        # head_num Node embeding_size
        if self.reduction == "concat":
            result = tf.concat(
                tf.split(result, self.head_num, axis=1), axis=-1)
            result = tf.squeeze(result, axis=1)
        else:
            result = tf.reduce_mean(result, axis=1)

        if self.act:
            result = self.activation(result)

        result._uses_learning_phase = True
        return result

    def compute_output_shape(self, input_shape):
        if self.reduction == "concat":

            return (None, self.att_embedding_size * self.head_num)
        else:
            return (None, self.att_embedding_size)




def GAT(nodes_number,feature_dim,num_class,num_layers=2,n_attn_heads = 8,att_embedding_size=8,dropout_rate=0.0,l2_reg=0.0,use_bias=True):
    X_in = Input(shape=(feature_dim,))
    A_in = Input(shape=(nodes_number,))
    h = X_in
    for _ in range(num_layers-1):
        h = GATLayer(att_embedding_size=att_embedding_size, head_num=n_attn_heads, dropout_rate=dropout_rate, l2_reg=l2_reg,
                                     activation="relu", use_bias=use_bias, )([h, A_in])

    h = GATLayer(att_embedding_size=num_class, head_num=1, dropout_rate=dropout_rate, l2_reg=l2_reg,
                                 activation=tf.nn.softmax, use_bias=use_bias, reduction='mean')([h, A_in])

    model = Model(inputs=[X_in, A_in], outputs=h)

    return model

In [11]:
from extra_keras_metrics import get_minimal_multiclass_metrics
model = GAT(
    nodes_number=cora.get_nodes_number(), 
    feature_dim=features.shape[1], 
    num_class=cora.get_node_types_number(), 
    num_layers=2,
    n_attn_heads=8, 
    att_embedding_size=8,
    dropout_rate=0.6, 
    l2_reg=2.5e-4, 
    use_bias=True
)

model.compile(
    optimizer="nadam",
    loss='categorical_crossentropy',
    weighted_metrics=get_minimal_multiclass_metrics()
)

In [13]:
model_input = [features, A]

model.fit(
    model_input,
    train_graph.get_one_hot_encoded_node_types(), 
    sample_weight=train_graph.get_node_ids_with_known_node_types_mask(), 
    batch_size=A.shape[0], 
    epochs=1000, 
    shuffle=False, 
    verbose=False,
    callbacks=[
        TqdmCallback(verbose=0)
    ]
)

0epoch [00:00, ?epoch/s]

KeyboardInterrupt: 